In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

train = pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\train_common_v2.0_1101.csv')
test = pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\test_common_v2.0_1101.csv')
target_r = pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\store_train (1).csv')
submit = pd.read_csv(r'C:\Users\leeya\OneDrive\Desktop\부캠 수업자료\Final\store_submission (2).csv')

#ID 제외하기
train_ft = train.iloc[:, 1:].copy() 
test_ft = test.iloc[:, 1:].copy()  
target = target_r.iloc[:, 1].copy() 

# 원핫인코딩
cols = train_ft.select_dtypes(include=['object']).columns
enc = OneHotEncoder(handle_unknown="ignore")
enc.fit(train_ft[cols])


train_encoded = pd.DataFrame(
    enc.transform(train_ft[cols]).toarray(),
    columns=enc.get_feature_names_out()
)
train_ft = pd.concat([train_ft.drop(columns=cols), train_encoded], axis=1)


test_encoded = pd.DataFrame(
    enc.transform(test_ft[cols]).toarray(),
    columns=enc.get_feature_names_out()
)
test_ft = pd.concat([test_ft.drop(columns=cols), test_encoded], axis=1)

# 피처 스케일링
scaler = MinMaxScaler()
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)


x_train, x_valid, y_train, y_valid = train_test_split(train_ft, target, test_size=0.2, random_state=42)

# 모델링
model = LogisticRegression(random_state=42, penalty='l2', solver='saga', max_iter=2000)
model.fit(x_train, y_train)
pred = model.predict(x_valid)


model.fit(train_ft, target)
test_pred = model.predict(test_ft)


id_column = submit["ID"]  # ID 열을 제출 파일에서 가져옴
pred_df = pd.DataFrame(test_pred, columns=["target"])
result_df = pd.concat([id_column.reset_index(drop=True), pred_df], axis=1)


result_df.to_csv("Logistic_test_HJ2.csv", index=False)


In [3]:
result_df.info

<bound method DataFrame.info of                ID  target
0          test_0     0.0
1          test_1     0.0
2          test_2     0.0
3          test_3     1.0
4          test_4     0.0
...           ...     ...
12220  test_12220     1.0
12221  test_12221     0.0
12222  test_12222     0.0
12223  test_12223     0.0
12224  test_12224     0.0

[12225 rows x 2 columns]>

In [2]:
print(result_df.isnull().sum())

ID        0
target    0
dtype: int64
